In [1]:
%load_ext autoreload
%autoreload 1
%matplotlib inline
# %aimport utils.remote_execute_util
import os
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import scikitplot as skplt
import joblib
from dotenv import load_dotenv
from azureml.core import Experiment
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from amlutils import aml_util as a_util
from amlutils import blobs_util as b_util
import mnist_util as m_util
load_dotenv()

True

In [3]:
ws = a_util.get_ws()

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


In [4]:
try:
    ds = b_util.get_ds(ws, 'dsmnist')
except Exception:
    ds = b_util.register_blob_ws(ws, 'dsmnist', 'mnist')

In [5]:
compute_target = a_util.prepare_remote_compute(ws, compute_name='gpucluster',
                                               compute_min_nodes=0,
                                               compute_max_nodes=1,
                                               compute_vm_size='STANDARD_NC6')
# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-12-22T14:25:36.140000+00:00', 'errors': None, 'creationTime': '2019-12-22T14:25:33.285697+00:00', 'modifiedTime': '2019-12-22T14:25:49.501592+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount'

In [23]:
# it's probably better to use the pre-built TensorFlow containers
from azureml.train.dnn import TensorFlow

script_params = {
    '--data-folder': ds.as_mount(),
    '--output-folder': 'outputs',
    '--batch-size': 50,
    '--first-layer-neurons': 300,
    '--second-layer-neurons': 100,
    '--learning-rate': 0.001
}

estimator = TensorFlow(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='mnist_tf_train.py',
                use_gpu=True,
                pip_requirements_file='requirements_remote_tf.txt',
                framework_version = '1.13'
                      )

In [24]:
experiment = Experiment(workspace=ws, name='exptfmnist')
run = experiment.submit(config=estimator)
run

Experiment,Id,Type,Status,Details Page,Docs Page
exptfmnist,exptfmnist_1577028215_41fe3510,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…

In [26]:
run.get_details()

{'runId': 'exptfmnist_1577028215_41fe3510',
 'target': 'gpucluster',
 'status': 'Completed',
 'startTimeUtc': '2019-12-22T15:24:30.352685Z',
 'endTimeUtc': '2019-12-22T15:26:43.61604Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '50f9a110-36d0-4102-b390-fe2707f617c2',
  'azureml.git.repository_uri': 'https://github.com/phonosync/amldemo.git',
  'mlflow.source.git.repoURL': 'https://github.com/phonosync/amldemo.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'e8272933d7c06cf801bbd13eb97e86631e2f6259',
  'mlflow.source.git.commit': 'e8272933d7c06cf801bbd13eb97e86631e2f6259',
  'azureml.git.dirty': 'True',
  'AzureML.DerivedImageName': 'azureml/azureml_bfc72ae7161f7e9ed53e4f75c2db836f',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'mnist_tf_train.py',
  'arguments': ['--data-

In [27]:
run.get_metrics()

{'Loss': [0.21885342204710467,
  0.0920572054372557,
  0.0638653649650223,
  0.05127834784238075,
  0.03892019454646288,
  0.03218105182268346,
  0.025724028131836576,
  0.022672735814066886,
  0.01800879143075837,
  0.01695795335330968,
  0.013503861620684532,
  0.01255319177486561,
  0.010277800037994672,
  0.009167975139537541,
  0.009733998803048262,
  0.006836234199568025,
  0.008639800970422372,
  0.007859517904660104,
  0.005695832867422708,
  0.005845642167257118],
 'Accuracy': [0.9345166640045742,
  0.9723000011841456,
  0.9806666694084804,
  0.9855000037451586,
  0.9886000047127406,
  0.9903000042339166,
  0.9925666702787082,
  0.9933500037093957,
  0.9944666701555253,
  0.9951666698853174,
  0.9963500025371711,
  0.9967666687568029,
  0.9972500023742517,
  0.9975833353400231,
  0.9975500018397967,
  0.998266668220361,
  0.9977666684488455,
  0.9981500013669332,
  0.9985500011841456,
  0.9985333346327145],
 'Final test loss': 0.16263739759627185,
 'Final test accuracy': 0.978

In [28]:
run.get_file_names()

['Accuracy vs Loss_1577028381.png',
 'azureml-logs/55_azureml-execution-tvmps_6e0737a629e4be65df6882812bab100d609dc2dce82b524e31a0976efaa70d3a_d.txt',
 'azureml-logs/65_job_prep-tvmps_6e0737a629e4be65df6882812bab100d609dc2dce82b524e31a0976efaa70d3a_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_6e0737a629e4be65df6882812bab100d609dc2dce82b524e31a0976efaa70d3a_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/138_azureml.log',
 'logs/azureml/azureml.log',
 'outputs/model/model.h5',
 'outputs/model/model.json']

In [ ]:
# create a model folder in the current directory
os.makedirs(os.path.join(os.getenv('OUTPUT_REMOTE'), 'mnist'), exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)